# Open-Roles Token Swap Using Marlowe Runtime\'s REST API

***Before running this notebook, you might want to use Jupyter's "clear output" function to erase the results of the previous execution of this notebook. That will make more apparent what has been executed in the current session.***

The token-swap contract example is a simple Marlowe contract that lets parties trade ada for a token. This example differs from Lesson 5 in that is employs an *open role* that allows *anyone* to become the buyer party in the contract. Namely, the identity of the buyer is unknown when the contract is instantiated on the blockchain. Only later does someone decide to participate as the buyer, at which time they receive the role token for the "buyer" party.

[A video works through this Jupyter notebook.](https://drive.google.com/file/d/1g17LKLJ4cyizfUJDrM44ChaNb6otOce0/view?usp=sharing)

You can ask questions about Marlowe in [the #ask-marlowe channel on the IOG Discord](https://discord.com/channels/826816523368005654/936295815926927390) or post problems with this lesson to [the issues list for the Marlowe Starter Kit github repository](https://github.com/input-output-hk/marlowe-starter-kit/issues).

In this demonsration we use Marlowe Runtime\'s REST API, served via `marlowe-web-server`, to run this contract on Cardano\'s `preprod` public testnet. Marlowe contracts may use either addresses or role tokens for authorization: here we use role tokens and we have Marlowe Runtime mint them.

## Overview of open roles in Marlowe

Although all the roles in a Marlowe contract must be specified when the contract is instantiated on the block-chain, the contract's role tokens need not be distributed to payment addresses at that time. Some roles can be left "open" for later assignment and distribution by sending them to a Plutus script called the open-roles validator when they are initially minted. That Plutus script holds the role token until a party makes an `IDeposit` or `IChoice` action as input to the contract; as they perform that action in a transaction, the open-role validator releases the role token to their wallet. This allows parties not identified by address at the time of the contract's instantiation to later participate in the contract.

### Details

One or more open roles can be included in a Marlowe contract by specifying them when Marlowe Runtime builds the transaction for instantiating the contract.

-   The creation transaction for the Marlowe contract does the following:
    -   Places the thread token in the contract's UTXO.
    -   Creates a UTXO for the open-roles validator.
        -   The `Datum` references the thread token that is placed in the contract's UTXO.
        -   The `Value` contains the open-role token.
-   The open-role assignment transaction does the following:
    -   Transfers the open-role token to the change address for the transaction.
    -   Verifies the validity of the transaction.
        -   The open-role validator ensures that `IDeposit` and/or `IChoice` actions for the open role are present in the input.
        -   The open-role validator ensures that the contract's UTXO contains the required thread token.
        -   The Marlowe semantics validator does its usual validation, including checking that the relevant inputs are authorized by the open-role token.

![Transactions involved in an open-roles Marlowe contract](open-roles-interaction.png)


#### Minting open roles

If role tokens are minted when the contract is created, specify the *thread role* required by the open-role validator and each *open role*. The example below has a thread role name `Thread`, and open role named `Buyer`, and a standard role named `Seller`. In the example below, the creation transaction will mint and send the thread role `Thread` to the Marlowe contract, the open role `Buyer` to the open-roles validator, and the role `Seller` to the seller's wallet address.

```yaml
roles:
  Thread:
    script: ThreadRole
  Buyer:
    script: OpenRole
  Seller: addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j
```

#### Pre-minted open roles

If role tokens have already been minted and are held in the wallet of the contract's creator, then specify the policy ID, the name of the thread token, and the names of the open-role tokens. In the example below, the creation transaction will send the thread role token `Thread` to the Marlowe contract and the open role `Buyer` to the open-roles validator. Any standard role tokens will have to be distributed to parties manually.

```yaml
roles:
  script: OpenRole
  policyId: ac5591a09baca1e04b5b2f086210cd1a9451032e02c1a327d13ee767
  threadRoleName: Thread
  openRoleNames:
    - Buyer
```

### Distribution of open-role tokens

The open-role validator authorizes the distribution of the open-role token that it holds to anyone who performs the contract's first action that takes the open role as input. After they have received the open-role token, they can use it as a role token for any future inputs to the contract or for any withdrawals from the role-payout validator address.

Furthermore, the open-role validator will only validate if the Marlowe contract holds the thread token specified when the open-role validator is created. This ties the open role to a particular Marlowe contract.


### Security restriction for open roles

Marlowe's prevention of double-satisfaction attacks requires that no external payments be made from a Marlowe contract if another Plutus script runs in the transaction. Thus, if an open-role is distributed in a transaction, the transaction cannot do `Pay` to parties or implicit payments upon `Close`. Typically, the distribution of an open-role in a Marlowe contract will be followed by a `Notify TrueObs` case so that further execution of the contract does not proceed in that transactions. External payments can be made in subsequent transactions.


## Example

The open-roles validator lets *anyone with a Cardano address* make a deposit or choice input for a contract that uses open roles. A simple use case for open roles is when a seller might offer a token for sale to anyone who pays the sales price.

1. The seller creates a contract that offers one `BearGarden` token for sale at the price of 8 ada.
2. The seller deposits the `BearGarden` token.
3. Anyone can join the contract by depositing the sale price of 8 ada into the contract. In that transaction, the open-role validator will pay them the `Seller` token so that they are now a party to the contract.
4. Anyone notifies the contract to complete its operation and pay the 8 ada to Marlowe's role-payout validator address for the benefit of the seller and one `BearGarden` for the benefit of the buyer. The `Notify TrueObs` is present because of the aforementioned security restriction.
5. The seller withdraws their 8 ada from the role-payout validator address.
6. The buyer withdraws their one `BearGarden` from the role-payout validator address.

In [Marlowe Playground](https://play.marlowe.iohk.io//), the contract looks like this in Blockly format.

![Example open-roles contract for Marlowe](open-roles-contract.png)

## Preliminaries

See [Preliminaries](../../docs/preliminaries.md) for information on setting up one's environment for using this tutorial.

The first step is to check we have all the required tools and environment variables available to the notebook. 

In [ ]:
export SCRIPTS=../../scripts
export KEYS=../../keys
source $SCRIPTS/check-tools-and-env.sh

Make sure you've also [setup and funded](../../setup/01-setup-keys.ipynb) the different parties
- Seller
    - `$KEYS/lender.address`: Cardano address for the seller
    - `$KEYS/lender.skey`: location of signing key file for the seller
- Buyer
    - `$KEYS/borrower.address`: Cardano address for the buyer
    - `$KEYS/borrower.skey`: location of signing key file for the buyer

### Seller address and funds

Check that an address and key has been created for the seller. If not, see "Creating Addresses and Signing Keys" in [Setup Keys](../../setup/01-setup-keys.ipynb#Creating-Addresses-and-Signing-Keys).

In [ ]:
SELLER_SKEY=$KEYS/lender.skey
SELLER_ADDR=$(cat $KEYS/lender.address)
echo "SELLER_ADDR = $SELLER_ADDR"

Check that the seller has at least ten hundred ada and the tokens.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$SELLER_ADDR"

One can view the address on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [ ]:
$SCRIPTS/cardano-scan-address.sh $SELLER_ADDR

### Buyer address and funds

Check that an address and key has been created for the buyer provider. If not, see "Creating Addresses and Signing Keys" in [Setup Keys](../../setup/01-setup-keys.ipynb#Creating-Addresses-and-Signing-Keys).

In [ ]:
BUYER_SKEY=$KEYS/borrower.skey
BUYER_ADDR=$(cat $KEYS/borrower.address)
echo "BUYER_ADDR = $BUYER_ADDR"

Check that the buyer has at least twenty ada.

In [ ]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BUYER_ADDR"

One can view the address on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [ ]:
$SCRIPTS/cardano-scan-address.sh $BUYER_ADDR

## Design the contract

### Specify the token to be offered

Fill in the policy ID and name of the token that will be offered for sale. If you need to mint a token for this example, see [Lesson 9. Minting Tokens for Marlowe Contracts](../09-minting).

In [ ]:
TOKEN_POLICY=8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
TOKEN_NAME=BearGarden
TOKEN_QUANTITY=1

### Set the deadlines for action.

In a real token swap the payment deadline might be weeks or months, instead of hours.

In [ ]:
DEPOSIT_DEADLINE=$((1000 * $(date -u -d "$date + 1 hour" +%s)))
PAYMENT_DEADLINE=$((1000 * $(date -u -d "$date + 2 hours" +%s)))
NOTIFY_DEADLINE=$((1000 * $(date -u -d "$date + 3 hours" +%s)))

### Create a JSON file containing the contract

In [ ]:
yaml2json << EOI > contract.json
when:
- case:
    party:
      role_token: Seller
    deposits: $TOKEN_QUANTITY
    of_token:
      currency_symbol: $TOKEN_POLICY
      token_name: $TOKEN_NAME
    into_account:
      role_token: Seller
  then:
    when:
    - case:
        party:
          role_token: Buyer
        deposits: 8000000
        of_token:
          currency_symbol: ''
          token_name: ''
        into_account:
          role_token: Seller
      then:
        pay: $TOKEN_QUANTITY
        token:
          currency_symbol: $TOKEN_POLICY
          token_name: $TOKEN_NAME
        to:
          account:
            role_token: Buyer
        from_account:
          role_token: Seller
        then:
          when:
          - case:
              notify_if: true
            then: close
          timeout: $NOTIFY_DEADLINE
          timeout_continuation: close
    timeout: $PAYMENT_DEADLINE
    timeout_continuation: close
timeout: $DEPOSIT_DEADLINE
timeout_continuation: close
EOI
jq . contract.json

## Transaction 1. Create the contract

Craft the request specifying the contract and its role tokens.

*Important:* An open-role contract requires one of the role tokens to be designated as a "thread" token and sent to the contract's UTXO. In this example we simply call the thread token `Thread`. Instead of assigning it a payment address, we assign it the JSON object `{"script": "ThreadRole"}`.

Similarly, each open role in the contract must be named. Below we just one one open role named `Buyer`. Instead of assigning it to a payment address, we assign it the JSON object `{"script" : "OpenRole"}`.

The special destination assignments for the thread and open roles are recognized by Marlowe Runtime as the cue for creating one or more open-role validators and sending the corresponding role tokens to the open-role validator address.

The `Seller` is just a standard role, so we assign it to be sent to the seller's payment address.

In [ ]:
yaml2json << EOI > request-1.json
version: v1
contract: $(cat contract.json)
roles:
  Thread:
    script: ThreadRole
  Buyer:
    script: OpenRole
  Seller: $SELLER_ADDR
minUTxODeposit: $((2 * 1000000))
metadata: {}
tags: {}
EOI
jq . request-1.json

A `HTTP` `POST` request to Marlowe Runtime\'s `/contracts` endpoint will build the creation transaction for a Marlowe contract. We provide it the JSON file containing the contract and tell it the `MIN_LOVELACE` value that we previously chose. Anyone could create the contract, but in this example the lender will be doing so, so we provide their address to fund the transaction and to receive the change from it.

In [ ]:
curl "$MARLOWE_RT_WEBSERVER_URL/contracts" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $SELLER_ADDR" \
  -d @request-1.json \
  -o response-1.json \
  -sS

Check to make sure the contract does not contain safety errors.

In [ ]:
jq .resource.safetyErrors response-1.json

Record the contract ID for future use.

In [ ]:
CONTRACT_ID="$(jq -r '.resource.contractId' response-1.json)"
echo "CONTRACT_ID = $CONTRACT_ID"

Extract the unsigned transaction from the response.

In [ ]:
jq '.resource.txBody' response-1.json > tx-1.unsigned
jq -r .type tx-1.unsigned

There are many ways to sign and submit Cardano transactions:
- `cardano-cli` at the command line
- `cardano-wallet` at the command line or as a REST service
- `cardano-hw-cli` for a hardware wallet at the command line
- a Babbage-compatible CIP-30 wallet in a web browser
- `marlowe-cli` at the command line

For convenience, here we use `marlowe-cli transaction submit`. One may have to wait a minute or so for the transactions to be confirmed on the blockchain.

In [ ]:
marlowe-cli transaction submit \
  --tx-body-file tx-1.unsigned \
  --required-signer $SELLER_SKEY \
  --timeout 600s

One can view the transaction on the Cardano and Marlowe explorer and see that the contract has been created and the parties have received their role tokens. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [ ]:
echo "Cardano Scan (low level)"
$SCRIPTS/cardano-scan-tx.sh "${CONTRACT_ID/#/%23}"
echo
echo "Marlowe Scan (high level)"
$SCRIPTS/marlowe-scan.sh "$CONTRACT_ID"

## Transaction 2. The seller deposits the `BearGarden` token.

Generate the JSON input for making the deposit.

In [ ]:
marlowe-cli input deposit \
  --deposit-party Seller \
  --deposit-account Seller \
  --deposit-amount 1 \
  --deposit-token 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.BearGarden \
  --out-file input-2.json
jq . input-2.json

Craft the request for Marlowe Runtime.

In [ ]:
yaml2json << EOI > request-2.json
version: v1
inputs: [$(cat input-2.json)]
metadata: {}
tags: {}
EOI
jq . request-2.json

Compute the URL for submitting transactions for the contract.

In [ ]:
CONTRACT_URL="$MARLOWE_RT_WEBSERVER_URL/`jq -r '.links.contract' response-1.json`"
echo "CONTRACT_URL = $CONTRACT_URL"

Request that Marlowe Runtime build the deposit transaction: the seller deposits their token into the contract using Marlowe Runtime\'s `HTTP` `POST` `/contract/{contractId}/transactions` endpoint.

In [ ]:
curl "$CONTRACT_URL/transactions" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $SELLER_ADDR" \
  -d @request-2.json \
  -o response-2.json \
  -sS

Extract the unsigned transaction from the response.

In [ ]:
jq '.resource.txBody' response-2.json > tx-2.unsigned
jq -r .type tx-2.unsigned

Submit the transaction.

In [ ]:
marlowe-cli transaction submit \
  --tx-body-file tx-2.unsigned \
  --required-signer $SELLER_SKEY \
  --timeout 600s

On can view the transaction on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [ ]:
$SCRIPTS/cardano-scan-tx.sh "$(jq -r .resource.transactionId response-2.json)"

## Transaction 3. The buyer deposits 8 ada and receives the `Buyer` role token.

Generate the JSON input for making the deposit.

In [ ]:
marlowe-cli input deposit \
  --deposit-party Buyer \
  --deposit-account Seller \
  --deposit-amount 8000000 \
  --out-file input-3.json
jq . input-3.json

Craft the request for Marlowe Runtime.

In [ ]:
yaml2json << EOI > request-3.json
version: v1
inputs: [$(cat input-3.json)]
metadata: {}
tags: {}
EOI
jq . request-3.json

Request that Marlowe Runtime build the deposit transaction. This time it is the buyer that is building and submitting the transaction.

In [ ]:
curl "$CONTRACT_URL/transactions" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $BUYER_ADDR" \
  -d @request-3.json \
  -o response-3.json \
  -sS

Extract the unsigned transaction from the response.

In [ ]:
jq '.resource.txBody' response-3.json > tx-3.unsigned
jq -r .type tx-3.unsigned

Submit the transaction.

In [ ]:
marlowe-cli transaction submit \
  --tx-body-file tx-3.unsigned \
  --required-signer $BUYER_SKEY \
  --timeout 600s

On can view the transaction on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [ ]:
$SCRIPTS/cardano-scan-tx.sh "$(jq -r .resource.transactionId response-3.json)"

## Transaction 4. Notify the contract to pay the `BearGarden` and 8 ada

Generate the JSON input for notifying the contract.

In [ ]:
marlowe-cli input notify \
  --out-file input-4.json
jq . input-4.json

Craft the request for Marlowe Runtime.

In [ ]:
yaml2json << EOI > request-4.json
version: v1
inputs: [$(cat input-4.json)]
metadata: {}
tags: {}
EOI
jq . request-4.json

Request that Marlowe Runtime build the notification transaction. Anyone can submit this transaction, but we choose to have the buyer submit it.

In [ ]:
curl "$CONTRACT_URL/transactions" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $BUYER_ADDR" \
  -d @request-4.json \
  -o response-4.json \
  -sS

Extract the unsigned transaction from the response.

In [ ]:
jq '.resource.txBody' response-4.json > tx-4.unsigned
jq -r .type tx-4.unsigned

Submit the transaction.

In [ ]:
marlowe-cli transaction submit \
  --tx-body-file tx-4.unsigned \
  --required-signer $BUYER_SKEY \
  --timeout 600s

On can view the transaction on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [ ]:
$SCRIPTS/cardano-scan-tx.sh "$(jq -r .resource.transactionId response-4.json)"

## Transaction 5. The buyer withdraws the `BearGarden` token.

Compute the URL for that transaction that paid to Marlowe's role-payout validator address.

In [ ]:
TX4_URL="$MARLOWE_RT_WEBSERVER_URL/$(jq -r .links.transaction response-4.json)"
echo "TX4_URL = $TX4_URL"

Fetch the list of payouts available for withdrawal via a HTTP `GET /contracts/{contract-id}/transactions/{transaction-id}`.

In [ ]:
curl -sS "$TX4_URL" \
| jq .resource.payouts \
> payouts.json
json2yaml payouts.json

Craft the request for Marlowe Runtime.

In [ ]:
yaml2json << EOI > request-5.json
payouts:
- $(jq -r '.[0].payoutId' payouts.json)
EOI
jq . request-5.json

Request that Marlowe Runtime build the withdrawal transaction vi a HTTP `POST /withdrawals`.

In [ ]:
curl "$MARLOWE_RT_WEBSERVER_URL/withdrawals" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $BUYER_ADDR" \
  -d @request-5.json \
  -o response-5.json \
  -sS

Extract the unsigned transaction from the response.

In [ ]:
jq '.resource.txBody' response-5.json > tx-5.unsigned
jq -r .type tx-5.unsigned

Submit the transaction.

In [ ]:
marlowe-cli transaction submit \
  --tx-body-file tx-5.unsigned \
  --required-signer $BUYER_SKEY \
  --timeout 600s

On can view the transaction on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [ ]:
$SCRIPTS/cardano-scan-tx.sh "$(jq -r .resource.withdrawalId response-5.json)"

## Transaction 6. The seller withdraws the 8 ada.

Craft the request for Marlowe Runtime.

In [ ]:
yaml2json << EOI > request-6.json
payouts:
- $(jq -r '.[1].payoutId' payouts.json)
EOI
cat request-6.json

Request that Marlowe Runtime build the withdrawal transaction.

In [ ]:
curl "$MARLOWE_RT_WEBSERVER_URL/withdrawals" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $SELLER_ADDR" \
  -d @request-6.json \
  -o response-6.json \
  -sS

Extract the unsigned transaction from the response.

In [ ]:
jq '.resource.txBody' response-6.json > tx-6.unsigned
jq -r .type tx-6.unsigned

Submit the transaction.

In [ ]:
marlowe-cli transaction submit \
  --tx-body-file tx-6.unsigned \
  --required-signer $SELLER_SKEY \
  --timeout 600s

On can view the transaction on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [ ]:
$SCRIPTS/cardano-scan-tx.sh "$(jq -r .resource.withdrawalId response-6.json)"

## Examine the wallets

Seller's wallet:

In [ ]:
cardano-cli query utxo --testnet-magic 1 --address "$SELLER_ADDR"

Buyer's wallet:

In [ ]:
cardano-cli query utxo --testnet-magic 1 --address "$BUYER_ADDR"